## Training a GAN on Fashion MNIST

#### Setting for hot reloading of modules

In [ ]:
%load_ext autoreload
%autoreload 2

#### Importing modules

In [ ]:
import lightning as L
import torchvision
import torchvision.transforms as transforms
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

from src.models.discriminator import Discriminator
from src.models.generator import Generator
from src.train.wrapper.gan_wrapper import GANWrapper
from src.utils.constants import Paths
from src.utils.helpers import detect_device, matplotlib_imshow

#### Setting up the data

In [ ]:
BATCH_SIZE = 512

#### Downloading and preparing the data

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.FashionMNIST(Paths.DATA_DIR, train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST(Paths.DATA_DIR, train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=False)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

#### Defining the model hyperparameters

In [ ]:
z_dim = 128
num_classes = len(classes)
generator_input_dim = z_dim + num_classes
input_channels = 1
discriminator_input_dim = input_channels + num_classes

#### Visualizing the data

In [ ]:
dataiter = iter(training_loader)
images, labels = next(dataiter)

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)
print('  '.join(classes[labels[j]] for j in range(4)))

#### Defining the model

In [ ]:
generator = Generator(generator_input_dim, input_channels)
discriminator = Discriminator(discriminator_input_dim)

gan_wrapper = GANWrapper(generator, discriminator, z_dim, num_classes, display_every_n_steps=100)

#### Defining the training parameters

In [ ]:
logger = TensorBoardLogger(Paths.LOGS_DIR, name='gan_training.logs')
checkpoint_callback = ModelCheckpoint(dirpath=Paths.MODEL_CHECKPOINT_DIR,
                                      filename='gan-{epoch:02d}-{val_generator_loss:.2f}', save_top_k=3,
                                      monitor='val_generator_loss')

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /logs/gan_training.logs

#### Training the model

In [ ]:
trainer = L.Trainer(default_root_dir=Paths.MODEL_CHECKPOINT_DIR, max_epochs=10000, callbacks=[checkpoint_callback],
                    logger=logger, accelerator=detect_device(), enable_checkpointing=True, log_every_n_steps=100)

trainer.fit(gan_wrapper, train_dataloaders=training_loader, val_dataloaders=validation_loader)